# Final Project

- Saige Belanger
    - (20951877)
- Dylan Faelker
    - (20960747)
- Ethan Liu
    - (20959615)
- Timothy Zheng
    - t54zheng (20939203)

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import warnings
from sklearn import linear_model
import statsmodels.api as sm
import scipy.stats as stats
from math import sqrt

from sklearn import preprocessing
from sklearn.impute import SimpleImputer
import datetime as dt

import os.path
import math

warnings.filterwarnings('ignore')

from multiprocess import Manager, cpu_count # You might have to change to multiprocessing if on windows
from multiprocess.pool import ThreadPool

# Factors
We start with an initial list of factors from the provided list of 50 Factors in the ML examples.

TODO: Increase our breadth of factors to the category chosen by downloading and creating them, then merging them 

https://wrds-www.wharton.upenn.edu/pages/get-data/compustat-capital-iq-standard-poors/compustat/north-america-daily/fundamentals-quarterly/

In [2]:
all_monthly_data = pd.read_sas("merged_df.sas7bdat", encoding = 'ISO-8859-1')

In [3]:
# Save all gvkeys - for WRDS Queries

# with open("gvkeys.txt", "w") as file:
#     for gvkey in set(all_monthly_data["gvkey"].dropna()):
#         file.write(f"{int(gvkey)},\n")

In [4]:
gvkey_permno_dict = {}
for gvkey in set(all_monthly_data["gvkey"].dropna()):
    permno = all_monthly_data[all_monthly_data["gvkey"] == gvkey]["permno"].dropna().iloc[0]
    gvkey_permno_dict[gvkey] = permno

In [5]:
all_monthly_data.drop(["ticker", "conm", "gvkey", "cusip", "naics", "gsubind"], axis=1, inplace=True) # We don't use these columns anyway, drop them

In [6]:
factors = ['IM', 'range_20', 'log_vol_dollar_20',
       'range_120', 'log_vol_dollar_120', 'xret_5', 'xret_10', 'xret_20',
       'xret_indsize_20', 'xret_indsize_std20', 'xret_40', 'xret_120',
       'xret_indsize_120', 'xret_indsize_std120', 'KDJ_20', 'deviation_pct20',
       'MoneyFlowIndex_20', 'RSI_20', 'KDJ_120', 'deviation_pct120',
       'MoneyFlowIndex_120', 'RSI_120', 'IV_capm', 'mdr', 'ami_3', 'beta_3y',
       'beta_5y', 'tail_2y', 'dp', 'leverage', 'BL', 'roe', 'roa',
       'profitability', 'sales_g_q', 'sales_g_ttm', 'op_income_g_q', 'ni_g_q',
       'op_income_g_ttm', 'ni_g_ttm', 'sue_NI', 'BM', 'AM', 'EP', 'SP',
       'roe_q', 'roa_q', 'Cto', 'pe_ttm', 'lag_log_size']

ret_cols = ['ret_f1', 'ret_f2', 'ret_f3', 'ret_f4', 'ret_f5', 'ret_f6', 
            'ret_f7', 'ret_f8', 'ret_f9', 'ret_f10', 'ret_f11', 'ret_f12']

In [7]:
non_data_cols = [x for x in all_monthly_data.columns if x not in factors and x not in ret_cols]
non_data_cols

['permno', 'yyyymm', 'monthid', 'PRC', 'VOL', 'RET', 'SHROUT']

# Adding New Factors
* When you add a factor, document it here: [link](https://docs.google.com/spreadsheets/d/1rs9633QSYLVY5Z5DoGNy3USP2MROGtqTIKcbLG68wpE/edit#gid=1579135478) and fill properly
* Download the data file, if it's too large add it to the drive
* Also download the other files that arent on github but on the drive before working on this part of the notebook
    * https://drive.google.com/drive/u/0/folders/1D1eIYlkNxNLfzHJLzkGeE9ymr7doXg_6

## IMPORTANT NOTE - FACTOR/RETURN TIME
- When adding factors make sure you add such that factor is reported at t-1, **RET** has **T** returns (in same row)
- This means you need to download data from the range **(1979-12 to 2019-11)**

***

- Treasury and CPI Rates: [Link](https://wrds-www.wharton.upenn.edu/pages/get-data/center-research-security-prices-crsp/annual-update/index-treasury-and-inflation/us-treasury-and-inflation-indexes/)
- Federal Reserve Data: [Link](https://wrds-www.wharton.upenn.edu/pages/get-data/federal-reserve-bank-reports/interest-rates/data/)
- SEC Filings: https://wrds-www.wharton.upenn.edu/pages/get-data/wrds-sec-analytics-suite/wrds-sec-filings-queries/list-of-filings-exhibits/
- Analyzed Data: https://wrds-www.wharton.upenn.edu/pages/get-data/wrds-sec-analytics-suite/wrds-sec-text-analysis/readability-and-sentiment/

**TBD**
- Other Factors: Downloaded from https://wrds-www.wharton.upenn.edu/pages/get-data/compustat-capital-iq-standard-poors/compustat/north-america-daily/fundamentals-quarterly/

In [8]:
sec_analytics = pd.read_sas("sec_filing_analysis_wrds.sas7bdat", encoding = 'ISO-8859-1')

In [9]:
sec_analytics["PERMNO"] = sec_analytics["GVKEY"].map(gvkey_permno_dict)

In [10]:
sec_analytics[["FDATE", "PERMNO", "FORM", "WORD_COUNT", "LM_MASTER_DICTIONARY_COUNT", "LM_NEGATIVE_COUNT",	"LM_POSITIVE_COUNT",	"LM_MODAL_WEAK_COUNT",	"LM_MODAL_STRONG_COUNT",	"LM_LITIGIOUS_COUNT",	"LM_UNCERTAINTY_COUNT",]]

# TODO Merge this into all_monthly_data, keeping in mind weird t+1 offset
# Also split by form?

,FDATE,PERMNO,FORM,WORD_COUNT,LM_MASTER_DICTIONARY_COUNT,LM_NEGATIVE_COUNT,LM_POSITIVE_COUNT,LM_MODAL_WEAK_COUNT,LM_MODAL_STRONG_COUNT,LM_LITIGIOUS_COUNT,LM_UNCERTAINTY_COUNT
0,2013-06-25,86594.0,10-K,260142.0,207916.0,2500.0,2387.0,458.0,814.0,4079.0,1585.0
1,2003-07-16,86594.0,4,107.0,131.0,1.0,0.0,0.0,0.0,1.0,0.0
2,2005-07-11,86594.0,4,82.0,60.0,0.0,0.0,0.0,0.0,1.0,0.0
3,2012-12-12,86594.0,S-8,2102.0,1762.0,13.0,4.0,4.0,1.0,78.0,8.0
4,2006-08-02,86594.0,4,61.0,44.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
906578,2019-05-13,18735.0,S-1,101132.0,92530.0,1603.0,1056.0,792.0,474.0,875.0,1375.0
906579,2019-05-02,18735.0,DRSLTR,2733.0,2492.0,16.0,17.0,3.0,6.0,47.0,34.0
906580,2019-05-02,18735.0,DRS/A,101883.0,93017.0,1590.0,1055.0,786.0,466.0,821.0,1367.0
906581,2019-04-25,18735.0,UPLOAD,2880.0,2580.0,28.0,22.0,10.0,4.0,48.0,34.0


In [11]:
sec_analytics[sec_analytics["PERMNO"] == 86594.0].sort_values(by="FDATE").head()

,CIK,FDATE,GVKEY,FORM,WORD_COUNT,COMPLEX_WORD_COUNT,ARI,COLEMAN_LIAU_INDEX,FLESCH_KINCAID_GRADE_LEVEL,FLESCH_READING_EASE,...,RIX,SMOG_INDEX,LM_MASTER_DICTIONARY_COUNT,LM_NEGATIVE_COUNT,LM_POSITIVE_COUNT,LM_MODAL_WEAK_COUNT,LM_MODAL_STRONG_COUNT,LM_LITIGIOUS_COUNT,LM_UNCERTAINTY_COUNT,PERMNO
173,0000056679,1998-08-18,118267,S-1,41172.0,10623.0,18.552631,17.113089,15.729576,27.472262,...,8.797905,17.744452,37549.0,322.0,411.0,206.0,220.0,338.0,417.0,86594.0
545,0000056679,1998-08-18,118267,S-1,41172.0,10623.0,18.552631,17.113089,15.729576,27.472262,...,8.797905,17.744452,37549.0,322.0,411.0,206.0,220.0,338.0,417.0,86594.0
544,0000056679,1998-09-04,118267,S-1/A,173837.0,39571.0,20.149471,14.896849,17.616691,26.687202,...,9.880636,18.672986,166236.0,2974.0,1202.0,857.0,792.0,6039.0,1014.0,86594.0
172,0000056679,1998-09-04,118267,S-1/A,173837.0,39571.0,20.149471,14.896849,17.616691,26.687202,...,9.880636,18.672986,166236.0,2974.0,1202.0,857.0,792.0,6039.0,1014.0,86594.0
512,0000056679,1998-09-22,118267,8-A12B,335.0,87.0,15.770003,17.433194,12.485647,38.695170,...,6.500000,15.688483,292.0,0.0,2.0,1.0,0.0,5.0,1.0,86594.0


In [18]:
tone_factors = ['10Q_tone_3m', '10Q_tone_6m', 
                '10K_tone_1y', 
                '8K_tone_1y',]
factors = factors + tone_factors

In [13]:
sec_analytics['yyyymm'] = (sec_analytics['FDATE'].astype('string').str[:4] + sec_analytics['FDATE'].astype('string').str[5:7]).astype('float64')
sec_analytics_subset_forms = sec_analytics[(sec_analytics['FORM'].isin(['10-K', '10-Q', '8-K']))]

sec_analytics_subset_forms['tone'] = (sec_analytics_subset_forms['LM_POSITIVE_COUNT'] - sec_analytics_subset_forms['LM_NEGATIVE_COUNT']) / sec_analytics_subset_forms['WORD_COUNT']

sec_analytics_subset_forms

,CIK,FDATE,GVKEY,FORM,WORD_COUNT,COMPLEX_WORD_COUNT,ARI,COLEMAN_LIAU_INDEX,FLESCH_KINCAID_GRADE_LEVEL,FLESCH_READING_EASE,...,LM_MASTER_DICTIONARY_COUNT,LM_NEGATIVE_COUNT,LM_POSITIVE_COUNT,LM_MODAL_WEAK_COUNT,LM_MODAL_STRONG_COUNT,LM_LITIGIOUS_COUNT,LM_UNCERTAINTY_COUNT,PERMNO,yyyymm,tone
0,0000056679,2013-06-25,118267,10-K,260142.0,78121.0,26.203477,22.244755,22.097121,-4.808299,...,207916.0,2500.0,2387.0,458.0,814.0,4079.0,1585.0,86594.0,201306.0,-0.000434
9,0000056679,2007-03-12,118267,10-Q,14978.0,3611.0,18.228793,15.566554,16.352541,26.504505,...,12671.0,115.0,110.0,29.0,26.0,48.0,114.0,86594.0,200703.0,-0.000334
11,0000056679,2012-09-05,118267,8-K,3111.0,758.0,17.460454,16.470280,15.351961,27.950419,...,2482.0,31.0,35.0,4.0,4.0,7.0,26.0,86594.0,201209.0,0.001286
14,0000056679,2005-02-23,118267,8-K,53640.0,13087.0,34.353908,14.788001,30.146895,-11.008120,...,51139.0,931.0,279.0,260.0,64.0,1877.0,346.0,86594.0,200502.0,-0.012155
17,0000056679,2006-06-07,118267,8-K,907.0,253.0,19.659327,17.735083,17.138851,19.699286,...,796.0,6.0,15.0,6.0,3.0,10.0,22.0,86594.0,200606.0,0.009923
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
906529,0001771515,2019-08-13,189579,8-K,3639.0,842.0,24.767273,17.189997,20.805662,14.122182,...,2970.0,71.0,23.0,8.0,7.0,16.0,46.0,18735.0,201908.0,-0.013190
906530,0001771515,2019-07-25,189579,8-K,165593.0,40606.0,51.771575,15.356026,43.395017,-45.079041,...,156053.0,2467.0,835.0,539.0,200.0,5025.0,858.0,18735.0,201907.0,-0.009855
906531,0001771515,2019-06-28,189579,8-K,898.0,222.0,18.958871,15.518174,16.994724,24.813588,...,759.0,10.0,4.0,0.0,0.0,20.0,2.0,18735.0,201906.0,-0.006682
906540,0001771515,2019-08-13,189579,10-Q,122460.0,33563.0,28.705696,23.824051,23.861851,-12.866557,...,94165.0,853.0,444.0,122.0,96.0,1209.0,841.0,18735.0,201908.0,-0.003340


In [14]:
def yyyymm_add(yyyymm, years, months):
    return yyyymm + ((years + math.floor(months / 12)) * 100) + (months % 12)

def yyyymm_sub(yyyymm, years, months):
    return yyyymm - ((years + math.floor(months / 12)) * 100) - (months % 12)

In [15]:
# Threaded Approach. Takes estimate >2h
def calc_tone_factors(permno):
    results = []

    for date in all_monthly_data['yyyymm'].unique():
        sentiments = {}
        # when there is no relevant data. Saves time
        if sec_analytics_subset_forms.loc[(permno == sec_analytics_subset_forms['PERMNO']) & (date > sec_analytics_subset_forms['yyyymm']) & (date <= yyyymm_add(sec_analytics_subset_forms['yyyymm'], 1, 0))].shape[0] == 0:
            sentiments = {"permno": permno, 
                            "yyyymm": date,
                            '10Q_tone_3m': 0,
                            '10Q_tone_6m': 0,
                            '10K_tone_1y': 0,
                            '8K_tone_1y': 0,
                            }
        elif sec_analytics_subset_forms.loc[(permno == sec_analytics_subset_forms['PERMNO']) & (date > sec_analytics_subset_forms['yyyymm']) & (date <= yyyymm_add(sec_analytics_subset_forms['yyyymm'], 0, 6))].shape[0] == 0:
            sentiments = {"permno": permno, 
                            "yyyymm": date,
                            '10Q_tone_3m': 0,
                            '10Q_tone_6m': 0,
                            '10K_tone_1y': sec_analytics_subset_forms.loc[('10-K' == sec_analytics_subset_forms['FORM']) & (permno == sec_analytics_subset_forms['PERMNO']) & (date > sec_analytics_subset_forms['yyyymm']) & (date <= yyyymm_add(sec_analytics_subset_forms['yyyymm'], 1, 0)), 'tone'].sum(),
                            '8K_tone_1y': sec_analytics_subset_forms.loc[('8-K' == sec_analytics_subset_forms['FORM']) & (permno == sec_analytics_subset_forms['PERMNO']) & (date > sec_analytics_subset_forms['yyyymm']) & (date <= yyyymm_add(sec_analytics_subset_forms['yyyymm'], 1, 0)), 'tone'].sum(),
                            }
        elif sec_analytics_subset_forms.loc[(permno == sec_analytics_subset_forms['PERMNO']) & (date > sec_analytics_subset_forms['yyyymm']) & (date <= yyyymm_add(sec_analytics_subset_forms['yyyymm'], 0, 3))].shape[0] == 0:
            sentiments = {"permno": permno, 
                            "yyyymm": date,
                            '10Q_tone_3m': 0,
                            '10Q_tone_6m': sec_analytics_subset_forms.loc[('10-Q' == sec_analytics_subset_forms['FORM']) & (permno == sec_analytics_subset_forms['PERMNO']) & (date > sec_analytics_subset_forms['yyyymm']) & (date <= yyyymm_add(sec_analytics_subset_forms['yyyymm'], 0, 6)), 'tone'].sum(),
                            '10K_tone_1y': sec_analytics_subset_forms.loc[('10-K' == sec_analytics_subset_forms['FORM']) & (permno == sec_analytics_subset_forms['PERMNO']) & (date > sec_analytics_subset_forms['yyyymm']) & (date <= yyyymm_add(sec_analytics_subset_forms['yyyymm'], 1, 0)), 'tone'].sum(),
                            '8K_tone_1y': sec_analytics_subset_forms.loc[('8-K' == sec_analytics_subset_forms['FORM']) & (permno == sec_analytics_subset_forms['PERMNO']) & (date > sec_analytics_subset_forms['yyyymm']) & (date <= yyyymm_add(sec_analytics_subset_forms['yyyymm'], 1, 0)), 'tone'].sum(),
                            }
        else:
            sentiments = {"permno": permno, 
                            "yyyymm": date,
                            '10Q_tone_3m': sec_analytics_subset_forms.loc[('10-Q' == sec_analytics_subset_forms['FORM']) & (permno == sec_analytics_subset_forms['PERMNO']) & (date > sec_analytics_subset_forms['yyyymm']) & (date <= yyyymm_add(sec_analytics_subset_forms['yyyymm'], 0, 3)), 'tone'].sum(),
                            '10Q_tone_6m': sec_analytics_subset_forms.loc[('10-Q' == sec_analytics_subset_forms['FORM']) & (permno == sec_analytics_subset_forms['PERMNO']) & (date > sec_analytics_subset_forms['yyyymm']) & (date <= yyyymm_add(sec_analytics_subset_forms['yyyymm'], 0, 6)), 'tone'].sum(),
                            '10K_tone_1y': sec_analytics_subset_forms.loc[('10-K' == sec_analytics_subset_forms['FORM']) & (permno == sec_analytics_subset_forms['PERMNO']) & (date > sec_analytics_subset_forms['yyyymm']) & (date <= yyyymm_add(sec_analytics_subset_forms['yyyymm'], 1, 0)), 'tone'].sum(),
                            '8K_tone_1y': sec_analytics_subset_forms.loc[('8-K' == sec_analytics_subset_forms['FORM']) & (permno == sec_analytics_subset_forms['PERMNO']) & (date > sec_analytics_subset_forms['yyyymm']) & (date <= yyyymm_add(sec_analytics_subset_forms['yyyymm'], 1, 0)), 'tone'].sum(),
                            }
        results.append(sentiments)
    
    return results

permnos = set(all_monthly_data["permno"])
with ThreadPool(cpu_count() - 1) as P:
    sentiments = P.map(calc_tone_factors, permnos)
    sentiments = [item for sublist in sentiments for item in sublist]
    sentiments = pd.DataFrame(sentiments)
    all_monthly_data = pd.merge(all_monthly_data, sentiments, on=["permno", "yyyymm"], how='outer')

In [52]:
all_monthly_data[all_monthly_data["permno"] == 86594.0].sort_values(by="monthid").head()

,permno,yyyymm,monthid,ticker,conm,gvkey,cusip,naics,gsubind,IM,...,ret_f3,ret_f4,ret_f5,ret_f6,ret_f7,ret_f8,ret_f9,ret_f10,ret_f11,ret_f12
382780,86594.0,199901.0,229.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.090476,0.109948,0.283019,-0.194853,0.223744,0.376866,-0.035230,0.016854,0.607735,-0.053265
382781,86594.0,199902.0,230.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.109948,0.283019,-0.194853,0.223744,0.376866,-0.035230,0.016854,0.607735,-0.053265,0.088929
382782,86594.0,199903.0,231.0,KFY,KORN FERRY,118267,500643200,561311,20202010,NaN,...,0.283019,-0.194853,0.223744,0.376866,-0.035230,0.016854,0.607735,-0.053265,0.088929,-0.206667
382783,86594.0,199904.0,232.0,KFY,KORN FERRY,118267,500643200,561311,20202010,NaN,...,-0.194853,0.223744,0.376866,-0.035230,0.016854,0.607735,-0.053265,0.088929,-0.206667,-0.109244
382784,86594.0,199905.0,233.0,KFY,KORN FERRY,118267,500643200,561311,20202010,NaN,...,0.223744,0.376866,-0.035230,0.016854,0.607735,-0.053265,0.088929,-0.206667,-0.109244,-0.183962


# Data Cleanup

In [12]:
# Inputation - as in ML Lecture 1

# Drop NA in all non-numerical columns
all_monthly_data.dropna(subset=non_data_cols, inplace=True)

grouped_med = all_monthly_data.groupby(by='monthid')
# the lambda function gets the median per group in the groupby object, and fills the NaN values with the median per group
imputed_grouped = grouped_med.transform(lambda y: y.fillna(y.median()))

# This line assigns the values of the medians 
all_monthly_data = all_monthly_data.assign(**imputed_grouped.to_dict(orient='series'))
all_monthly_data.dropna(inplace=True)

In [7]:
# Filtering data by min price and min market share for each year

# Commenting out for runtime - does not drop any rows

# all_monthly_data['yyyy'] = all_monthly_data['yyyymm'].astype(str).str[:4]
# all_monthly_data['MKTSHR'] = all_monthly_data['PRC'] * all_monthly_data['SHROUT'] * 1_000

# to_drop_indices = []

# for permno in all_monthly_data.permno.unique():
#     for year in all_monthly_data['yyyy'].unique():
#         mask = (all_monthly_data['permno'] == permno) & (all_monthly_data['yyyy'] == year)
#         if all_monthly_data[mask].shape[0] != 0 != 0 and (all_monthly_data[mask]['MKTSHR'].iloc[0] < 100_000_000 or all_monthly_data[mask]['PRC'].iloc[0] <= 5):
#             to_drop_indices += list(all_monthly_data[mask].index)
# all_monthly_data.drop(to_drop_indices, inplace=True)

In [13]:
# Winsorizing factors--should winsorize the variables by quarter
for column in factors:
    for date in set(list(all_monthly_data["monthid"])):
        mask = (all_monthly_data["monthid"] == date)
        
        std = all_monthly_data[column][mask].std()
        mean = all_monthly_data[column][mask].mean()

        upper = mean + 3 * std
        lower = mean - 3 * std
        
        all_monthly_data[column][mask].clip(lower, upper, inplace= True)

## Factor Code

In [15]:
ff4_factors = pd.read_sas("ff4_factors.sas7bdat", encoding = 'ISO-8859-1')
ff4_factors["monthid"] = ff4_factors.index + 1
ff4_factors.head()

,DATEFF,SMB,HML,MKTRF,RF,UMD,monthid
0,1980-01-31,0.0162,0.0175,0.0551,0.0080,0.0755,1
1,1980-02-29,-0.0185,0.0061,-0.0122,0.0089,0.0788,2
2,1980-03-31,-0.0664,-0.0101,-0.1290,0.0121,-0.0955,3
3,1980-04-30,0.0105,0.0106,0.0397,0.0126,-0.0043,4
4,1980-05-30,0.0213,0.0038,0.0526,0.0081,-0.0112,5


In [16]:
dates = [int(x) for x in sorted(list(set(list(all_monthly_data["yyyymm"]))))]
dates[0], dates[-1]

(198004, 201912)

In [17]:
monthids = [int(x) for x in sorted(list(set(list(all_monthly_data["monthid"]))))]
monthids[0], monthids[-1], len(monthids)

(4, 480, 477)

In [18]:
testing_range = monthids[0:2*(len(monthids) // 3)]
validation_range = monthids[2 * len(monthids) // 3:]

# Validate that ranges have correct ratios
len(testing_range) / len(monthids), len(validation_range) / len(monthids), len(testing_range) + len(validation_range)

(0.6666666666666666, 0.3333333333333333, 477)

## Testing Factors

In [20]:
model_factors = ['IM', 'range_20', 'log_vol_dollar_20',
       'range_120', 'log_vol_dollar_120', 'xret_5', 'xret_10', 'xret_20',
       'xret_indsize_20', 'xret_indsize_std20', 'xret_40', 'xret_120',
       'xret_indsize_120', 'xret_indsize_std120', 'KDJ_20', 'deviation_pct20',
       'MoneyFlowIndex_20', 'RSI_20', 'KDJ_120', 'deviation_pct120',
       'MoneyFlowIndex_120', 'RSI_120', 'IV_capm', 'mdr', 'ami_3', 'beta_3y',
       'beta_5y', 'tail_2y', 'dp', 'leverage', 'BL', 'roe', 'roa',
       'profitability', 'sales_g_q', 'sales_g_ttm', 'op_income_g_q', 'ni_g_q',
       'op_income_g_ttm', 'ni_g_ttm', 'sue_NI', 'BM', 'AM', 'EP', 'SP',
       'roe_q', 'roa_q', 'Cto', 'pe_ttm', 'lag_log_size',]
model_factors = model_factors + tone_factors

In [20]:
all_monthly_data = pd.merge(ff4_factors, all_monthly_data, on="monthid")

## [m, n, l] model for Fama-MacBeth Double Regression
We will use the technique employed during Assignment 2, utilizing a 36-month lookback for factor data to generate our betas (**First Stage**)
* For period $t_i$, we will use data starting at $t_{i-36} ... t_{i-1}$ if available. Worst case we look for 12 prior samples.

In [21]:
permnos = set(all_monthly_data["permno"])

In [22]:
!pip install multiprocess

In [24]:
from multiprocess import Manager, cpu_count # You might have to change to multiprocessing if on windows
from multiprocess.pool import ThreadPool

In [25]:
# Threaded Approach
def add_betas(permno):
    results = []
    for (i, monthid) in enumerate(testing_range): 
        window = set(testing_range[max(0, i-35):i+1]) # t_(i-36) to t_(i-1) returns. Compare to t_i returns
        window_data = all_monthly_data[(all_monthly_data["permno"] == permno) & (all_monthly_data["monthid"].isin(window))]
        
        if len(window_data) < 12:
            continue

        explanatory_vars = window_data[model_factors + ["monthid"]]
        explanatory_vars.sort_values(by="monthid", inplace=True)
        explanatory_vars.set_index("monthid", inplace=True)
    
        explained_var = window_data[["monthid", "RET"]] # Since factors are from t-1
        explained_var.sort_values(by="monthid", inplace=True)
        explained_var.set_index("monthid", inplace=True)
        
        model = linear_model.LinearRegression().fit(explanatory_vars, 
                                                    explained_var["RET"])
        
        results.append({"monthid": monthid, 
                        "permno": permno, 
                        "RET": explained_var["RET"].iloc[-1], 
                    **{f"{factor}": model.coef_[i] for i, factor in enumerate(model_factors)}
                       })  
    return results

# UNCOMMENT THIS AND ADD TO LINE BELOW INSTEAD OF `permnos`
# FOR DEVELOPMENT - THIS CODE BLOCK TAKES LIKE 30 MINS TO RUN

# smaller_permno_list = list(permnos)[:10]

# Only compute if not in files (delete local copy of file if code above if modifying factors or code above)
if os.path.isfile("first_stage_df.csv"):
    first_stage_df = pd.read_csv("first_stage_df.csv", index_col=0)
else:
    # Runs once basically
    summary_results = []
    with ThreadPool(cpu_count() - 1) as P:
        summary_results = P.map(add_betas, permnos)
        summary_results = [item for sublist in summary_results for item in sublist]
        first_stage_df = pd.DataFrame(summary_results)
        
        # Save first stage df for easy loading
        first_stage_df.to_csv("first_stage_df.csv")
first_stage_df

,monthid,permno,RET,IM,range_20,log_vol_dollar_20,range_120,log_vol_dollar_120,xret_5,xret_10,...,sue_NI,BM,AM,EP,SP,roe_q,roa_q,Cto,pe_ttm,lag_log_size
0,15,49154.0,0.217742,-0.177922,-0.017326,-0.404999,0.001008,0.094084,0.162161,0.357050,...,-0.103432,-0.162681,0.226897,-0.000786,0.194951,0.006577,-0.006378,-0.030747,0.038512,0.001883
1,16,49154.0,0.037528,-0.166218,-0.016434,-0.357248,0.001120,0.151541,0.173540,0.401040,...,-0.177359,-0.161623,0.201678,-0.000921,0.181788,0.015621,0.009557,0.075181,-0.006746,0.006358
2,17,49154.0,0.040426,-0.307559,-0.019532,-0.228961,0.001087,0.254345,0.228074,0.380108,...,-0.020192,-0.188044,0.077337,-0.001099,0.045979,0.010966,0.005602,0.043739,0.036017,-0.035808
3,18,49154.0,-0.113402,-0.352726,-0.020887,-0.213851,0.000877,0.238876,0.219848,0.344600,...,-0.074884,-0.184293,0.155087,-0.001629,0.063590,0.019056,0.004786,0.047677,0.020254,-0.022703
4,19,49154.0,0.046512,-0.352828,-0.020891,-0.213848,0.000877,0.238716,0.219853,0.344728,...,-0.074928,-0.184306,0.155164,-0.001633,0.063437,0.019088,0.004799,0.047794,0.020266,-0.022781
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
218367,317,81917.0,-0.021578,0.489096,10.687724,-0.466841,3.707940,0.683761,4.434208,-8.056764,...,-0.082417,-6.634807,1.790930,-2.328958,2.991200,19.291439,12.913282,-3.652366,-0.056563,2.546052
218368,318,81917.0,0.041499,0.603172,19.804879,-0.496944,12.429544,0.671798,2.554879,-10.596175,...,0.391672,-4.027635,-2.818458,-2.271254,3.142411,-3.350635,0.841312,2.354425,-0.205835,2.699524
218369,319,81917.0,0.008880,0.605449,20.417885,-0.482653,11.000336,0.662651,3.324765,-9.955248,...,0.477191,-3.633941,-2.279511,-1.578396,3.011760,-6.184711,-0.532376,2.476481,-0.178907,2.775979
218370,320,81917.0,0.063117,-0.228668,31.117055,-0.507347,1.068001,0.524543,6.665282,-2.317012,...,-0.102278,-3.579858,3.059718,-0.709997,-1.842430,-15.868266,-7.148372,3.467508,-0.043580,0.209534


In [26]:
# Second stage regression
lambdas = {"monthid": []}
for factor in model_factors:
    lambdas[f"{factor}"] = []
    
for monthid in testing_range:
    monthid_returns = first_stage_df.loc[first_stage_df["monthid"] == monthid]

    # If empty
    if monthid_returns.empty:
        continue
    
    explanatory_vars = monthid_returns[model_factors + ["permno"]]
    explanatory_vars.sort_values(by="permno", inplace=True)
    explanatory_vars.set_index("permno", inplace=True)

    explained_var = monthid_returns[["permno", "RET"]]
    explained_var.sort_values(by="permno", inplace=True)
    explained_var.set_index("permno", inplace=True)
    
    model = linear_model.LinearRegression(n_jobs=len(model_factors)).fit(explanatory_vars, 
                                                                         explained_var["RET"])

    lambdas["monthid"].append(monthid)

    for (i, factor) in enumerate(model_factors):
        lambdas[factor].append(model.coef_[i])

In [27]:
second_stage_df = pd.DataFrame(lambdas)
second_stage_df

,monthid,IM,range_20,log_vol_dollar_20,range_120,log_vol_dollar_120,xret_5,xret_10,xret_20,xret_indsize_20,...,sue_NI,BM,AM,EP,SP,roe_q,roa_q,Cto,pe_ttm,lag_log_size
0,15,-0.035745,-0.571039,-0.061605,-2.283169,-0.029881,-0.108176,-0.104386,0.149525,-0.212826,...,-0.041983,0.026534,0.015382,-0.889307,0.021992,-0.529210,3.357724,-0.393992,-0.057799,0.079260
1,16,0.001652,-0.048982,0.057267,-0.534534,0.026931,0.026047,0.052760,-0.061393,0.012480,...,-0.040266,0.112582,-0.021534,-0.343119,-0.059704,-0.261934,-0.299428,0.035351,0.346079,0.070998
2,17,-0.021220,-0.419154,-0.066717,0.775655,0.019364,0.004047,0.003952,-0.009959,-0.019862,...,-0.034248,0.083042,-0.006422,-0.377669,-0.092692,-0.554356,0.889185,0.121707,0.001101,0.029692
3,18,-0.021485,0.345735,-0.060198,-0.670695,0.009679,-0.006652,-0.001722,-0.016863,0.037531,...,0.006454,0.016666,0.015354,0.207710,0.023645,0.106906,-0.467556,-0.064735,-0.136481,0.051276
4,19,-0.030211,0.044757,-0.012854,0.110476,0.037228,0.007549,0.012964,0.001766,0.028058,...,0.018864,-0.043275,0.024022,-0.027254,-0.015462,0.399238,-0.343996,0.002513,0.093189,0.000007
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
302,317,-0.002052,0.000121,0.000777,0.000112,-0.001494,-0.000959,-0.002133,-0.003470,-0.003873,...,0.015909,-0.000528,0.000129,-0.000113,0.000229,-0.000227,-0.000908,0.000146,0.000216,-0.001342
303,318,0.002857,0.000061,-0.003151,0.000102,-0.002905,0.000159,0.000470,0.001164,0.000438,...,0.002612,0.000112,0.000263,0.000272,0.000095,-0.000557,0.000446,0.000588,0.012407,0.003445
304,319,-0.007282,0.000354,0.014312,0.000193,0.006701,0.000748,-0.001218,0.001316,0.001158,...,0.001495,-0.000300,-0.001778,-0.000241,0.000448,0.000159,-0.000518,-0.000001,-0.010411,-0.002123
305,320,0.000738,0.000232,-0.000996,0.000437,0.002546,-0.000941,-0.000438,-0.000859,0.002232,...,-0.013841,0.000209,0.000460,0.000309,-0.000146,0.000065,0.000935,0.000015,0.000445,-0.004901


In [28]:
# Get p values
p_value_dict = {"factor": [], "p-value": []}
for factor in model_factors:
    lambdas = second_stage_df[factor]
    ttest = stats.ttest_ind(lambdas, np.zeros(len(lambdas))) # Compare to see if any lambdas are significantly different from zero
    p_value_dict['factor'].append(factor)
    p_value_dict['p-value'].append(ttest[1])

results_df = pd.DataFrame.from_dict(p_value_dict, orient='index')
results_df.round(2).T.sort_values(by="p-value")

,factor,p-value
49,lag_log_size,0.001659
41,BM,0.03193
4,log_vol_dollar_120,0.057241
23,mdr,0.09065
32,roa,0.099273
16,MoneyFlowIndex_20,0.101928
37,ni_g_q,0.128012
28,dp,0.154698
9,xret_indsize_std20,0.160873
13,xret_indsize_std120,0.181982


# TODO
use these results to determine which factors to keep (among other considerations like cross-correlation, if they are in the same category, etc)

# Machine Learning

frick

# Performance Analysis

In [23]:
def total_ret(port_ret):
    return port_ret.sum()
    # return np.prod(port_ret + 1) - 1

def tracking_error(port_ret, bench_ret):
    return (port_ret - bench_ret).std()

def information_ratio(port_ret, bench_ret):
    return (total_ret(port_ret) - total_ret(bench_ret)) / tracking_error(port_ret, bench_ret)

def sharpe_ratio(port_ret, rf_ret):
    return information_ratio(port_ret, rf_ret)

def sharpe_ratio(port_xret):
    return total_ret(port_xret) / port_xret.std()

In [27]:
# Write Permnos - for WRDS Queries

# with open("permnos.txt", "w") as file:
#     for permno in permnos:
#         file.write(f"{int(permno)},\n")